#### _______IMPORTS Y AUTENTICACIONES_______

In [1]:
# IMPORTS
import requests
import pandas as pd
import csv

# AUTENTICACIÓN
api_key = '3784874dc3142503331b4556466b8965'  
base_url = 'http://ws.audioscrobbler.com/2.0/'


### _________LECTURA CSV DESDE SPOTIPY Y CONVERSIÓN EN LISTA_________

In [2]:
lista_artistas = pd.read_csv("artistas_v2.csv")
nombres_artistas = lista_artistas["artist"].tolist() 

COMPROBACIONES

In [3]:
len(nombres_artistas)

334

# ________CONSULTA DE BIO, ARTISTAS SIMILARES, REPRODUCCIONES, TAGS Y TOUR________

In [ ]:
artistasFM = []  # Lista donde guardaremos los diccionarios de cada artista

for artist in nombres_artistas:   # Queremos que para cada artista de la lista, recorra cada uno de ellos y obtengamos su información
    params_info = {               # Son los parámetros de la petición de la API
        'method': 'artist.getinfo',
        'artist': artist,
        'api_key': api_key,
        'format': 'json'} 
    try:                                 
        response_info = requests.get(base_url, params=params_info)  # Response nos indica si hay algún error en la petición
        response_info.raise_for_status()         # Si la respuesta tiene un código de error (tipo 400), se va al except
                                
        data_json = response_info.json()  # Extraemos los datos de la API en formato JSON
        nombre_artista = data_json.get('artist', {})  # Si no encuentra el dato, nos devuelve un diccionario vacío
        
        # Reproducciones y oyentes
        stats = nombre_artista.get('stats', {})       # stats es el sub-diccionario donde suele venir info como playcount y listeners 
        playcount = stats.get('playcount', 0)         # Total de reproducciones. Devolveremos 0 si la clave no existe
        listeners = stats.get('listeners', 0)         # Número de oyentes. Devolveremos 0 si la clave no existe
        
        # Biografía resumida
        biografia = nombre_artista.get('bio', {}).get('summary', '')  # Resumen de la biografía. Si no existe, devuelve cadena vacía
        
        # Artistas similares
        artistas_similares = []
        similar_artists = nombre_artista.get('similar', {}).get('artist', [])   # Seguimos el mismo patrón
        for s in similar_artists:
            artistas_similares.append(s['name'])
        
        # Tags del artista (como otros géneros asociados)
        tags_artist = []
        tags_list = nombre_artista.get('tags', {}).get('tag', [])
        for t in tags_list:
            tags_artist.append(t['name'])
        
        # Estado de tour
        ontour = nombre_artista.get('ontour', '0')  # 0 = no está de gira, 1 = sí
        
        # Creamos un diccionario con toda la información del artista
        artistasFM.append({      
            'Nombre Artista': artist,
            'Reproducciones': playcount,
            'Oyentes': listeners,
            'Biografia': biografia,
            'Artistas Similares': ", ".join(artistas_similares), # convierte la lista en una cadena separada por comas
            'Tags': ", ".join(tags_artist),
            'Ontour': ontour})      
        
    except requests.exceptions.RequestException as excep:   # Si ocurre un error de red o de la API, lo imprime y continúa
        print(f"El error para el artista {artist}: {excep}")


### _________CONVERSIÓN A DATAFRAME_________

In [5]:
df_artistas = pd.DataFrame(artistasFM)
df_artistas

,Nombre Artista,Reproducciones,Oyentes,Biografia,Artistas Similares,Tags,Ontour
0,The Black Keys,191878192,3214875,The Black Keys are an alternative rock group ...,"Dan Auerbach, The Arcs, Black Pistol Fire, The...","blues rock, blues, rock, indie rock, indie",1
1,Three Days Grace,167840674,3652299,"Three Days Grace is a Canadian rock band, form...","Saint Asonia, Breaking Benjamin, Seether, My D...","alternative rock, rock, hard rock, alternative...",1
2,Bon Iver,228256135,3427356,\nBon Iver is a Grammy Award winning folk ban...,"Justin Vernon, Volcano Choir, Sufjan Stevens, ...","folk, indie, singer-songwriter, acoustic, Lo-Fi",0
3,Mary J. Blige,32620515,2242351,"Mary Jane Blige (born January 11, 1971 in Bron...","Faith Evans, Toni Braxton, Monica, Brandy, Xscape","rnb, soul, female vocalists, Hip-Hop, urban",0
4,Joe Cocker,19263562,1534486,"John Robert ""Joe"" Cocker (20 May 1944 – 22 Dec...","Chris Rea, Eric Clapton, Rod Stewart, Leon Rus...","classic rock, blues, rock, blues rock, soul",0
...,...,...,...,...,...,...,...
329,Pete Townshend,3567769,348281,Peter Dennis Blandford Townshend (born 19 May ...,"The Who, John Entwistle, Roger Daltrey, The Ro...","classic rock, rock, singer-songwriter, genius,...",0
330,One Direction,288929569,3049709,One Direction (commonly abbreviated to 1D) wer...,"Louis Tomlinson, Niall Horan, Harry Styles, Li...","pop, british, boyband, one direction, X factor",0
331,Café Quijano,966217,98627,Café Quijano is a Spanish Pop group. It is mad...,"Pereza, Estopa, Joaquín Sabina, Celtas Cortos,...","spanish, pop, Spanish Rock, rock, Latin Rock",0
332,Antonio Orozco,1407509,116980,Antonio Orozco is a Spanish singer-songwriter ...,"Manuel Carrasco, Conchita, Melendi, El Arrebat...","spanish, pop, Flamenco, spanish pop, antonio o...",0


### _________CONVERSIÓN A CSV_________

In [6]:
df_artistas.to_csv('bio_artistas_v2.csv',index=False)